# Webscraping bvc.cv with beautifulsoup4 

### Import libraries and make request to bvc historico de cotações page

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import plotly.express as px
from pathlib import Path

html_text = requests.get('https://bvc.cv/pagina/historico-cotacoes-bvc-62').text
soup = BeautifulSoup(html_text, 'lxml')
epfds = soup.find('tbody', class_= 'text-right').find_all('td')

list = []

for index, epfd in enumerate(epfds):
    list.append(epfd.text)

#Convert list to list of lists (entidade, preco_fecho, dates)
list_epfd = [list[i:i+3] for i in range(0, len(list), 3)]
#Convert list of list to dataframe
df_hist_cot = pd.DataFrame(list_epfd, columns=['entidade', 'preco_fecho', 'dates'])



In [2]:
print(df_hist_cot.head())
print(df_hist_cot.info())

  entidade preco_fecho       dates
0      SCT        5750  2024-06-30
1      ENA       11000  2024-06-30
2    CAIXA        7180  2024-06-30
3      BCA        7100  2024-06-30
4      SCT        8910  2024-05-31
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   entidade     820 non-null    object
 1   preco_fecho  820 non-null    object
 2   dates        820 non-null    object
dtypes: object(3)
memory usage: 19.3+ KB
None


### Convert data types

In [3]:
df_hist_cot = df_hist_cot.astype({'preco_fecho': float})
df_hist_cot['dates'] = pd.to_datetime(df_hist_cot['dates'])
print(df_hist_cot.info())
print(df_hist_cot)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 820 entries, 0 to 819
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   entidade     820 non-null    object        
 1   preco_fecho  820 non-null    float64       
 2   dates        820 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 19.3+ KB
None
    entidade  preco_fecho      dates
0        SCT       5750.0 2024-06-30
1        ENA      11000.0 2024-06-30
2      CAIXA       7180.0 2024-06-30
3        BCA       7100.0 2024-06-30
4        SCT       8910.0 2024-05-31
..       ...          ...        ...
815      BCA       3900.0 2006-02-08
816      SCT       6500.0 2006-02-01
817      BCA       4000.0 2006-01-16
818      BCA       3500.0 2006-01-12
819      BCA       3500.0 2005-12-21

[820 rows x 3 columns]


## Clean the data

Check for duplicate dates

In [4]:
#find duplicate rows across based off entidade and dates columns
duplicateRows = df_hist_cot[df_hist_cot.duplicated(['entidade', 'dates'], keep=False)]
duplicateRows

,entidade,preco_fecho,dates
105,BCA,1920.0,2021-12-28
106,BCA,1900.0,2021-12-28
192,ENA,2772.0,2019-05-02
193,ENA,2772.0,2019-05-02


ENA has 2 same preco_fecho's for the same date. This causes the preco_fecho of that day to be the sum of the duplicate values. <br>
BCA has 2 diferent preco_fecho on same date. This causes the same issue, the preco_fecho of that day will be the sum of the duplicate values.

In [5]:
# So I will drop a an entirely duplicate row, this will solve the issue for ENA.

df_hist_cot = df_hist_cot[~df_hist_cot.duplicated()]


Check for duplicate rows again

In [6]:
duplicateRows = df_hist_cot[df_hist_cot.duplicated(['entidade', 'dates'], keep=False)]
duplicateRows

,entidade,preco_fecho,dates
105,BCA,1920.0,2021-12-28
106,BCA,1900.0,2021-12-28


The second row with the date of 2021-12-28 should be in fact 2021-12-29. That's obviously not right so will edit the date manualy. 

In [7]:
#df_hist_cot[df_hist_cot.duplicated(['entidade', 'dates'], keep='last')].index

df_hist_cot.loc[df_hist_cot[df_hist_cot.duplicated(['entidade', 'dates'], keep='last')].index, 'dates'] = '2021-12-29'


In [8]:
#check if dates are good
print(df_hist_cot[df_hist_cot['dates'] == '2021-12-29'])
print(df_hist_cot[df_hist_cot['dates'] == '2021-12-28'])

    entidade  preco_fecho      dates
104    CAIXA       2735.0 2021-12-29
105      BCA       1920.0 2021-12-29
    entidade  preco_fecho      dates
106      BCA       1900.0 2021-12-28


### Start EDA

In [9]:
df_hist_cot.describe()

,preco_fecho
count,819.000000
mean,4379.028083
std,1943.385540
min,1428.000000
25%,3000.000000
50%,3900.000000
75%,6000.000000
max,14950.000000


### Create dataframes for each entidade

In [10]:
df_SCT = df_hist_cot[df_hist_cot['entidade'] == 'SCT']
print('_________SCT_________')
print(df_SCT.describe())

df_BCA = df_hist_cot[df_hist_cot['entidade'] == 'BCA']
print('_________BCA_________')
print(df_BCA.describe())

df_CAIXA = df_hist_cot[df_hist_cot['entidade'] == 'CAIXA']
print('_________CAIXA_________')
print(df_CAIXA.describe())

df_ENA = df_hist_cot[df_hist_cot['entidade'] == 'ENA']
print('_________ENA_________')
print(df_ENA.describe())

_________SCT_________
        preco_fecho
count    122.000000
mean    5901.024590
std     1968.417363
min     3000.000000
25%     4892.500000
50%     5845.000000
75%     6500.000000
max    14950.000000
_________BCA_________
       preco_fecho
count   280.000000
mean   3008.946429
std     722.306764
min    1428.000000
25%    2837.500000
50%    3100.000000
75%    3200.000000
max    7224.000000
_________CAIXA_________
       preco_fecho
count    88.000000
mean   3215.477273
std    1163.678571
min    2070.000000
25%    2497.500000
50%    2910.000000
75%    3225.000000
max    7180.000000
_________ENA_________
        preco_fecho
count    329.000000
mean    5291.890578
std     1877.065602
min     2000.000000
25%     4000.000000
50%     4620.000000
75%     6795.000000
max    13680.000000


In [11]:
filepath = Path('data/bvc_quotes_history.csv')
df_hist_cot.to_csv(filepath, index=False)